In [2]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import sys

def clean_csv_file(file_path, output_folder):
    """
    Clean a single CSV file and save the cleaned version
    
    Parameters:
    file_path (str): Path to the input CSV file
    output_folder (str): Path to save the cleaned CSV file
    """
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        original_columns = df.columns.tolist()
        
        # Store original row count
        original_rows = len(df)
        
        # Basic cleaning operations
        
        # 1. Remove duplicates
        df = df.drop_duplicates()
        
        # 2. Strip whitespace from string columns
        string_columns = df.select_dtypes(include=['object']).columns
        for col in string_columns:
            df[col] = df[col].str.strip() if df[col].dtype == 'object' else df[col]
        
        # 3. Standardize text case in string columns
        for col in string_columns:
            df[col] = df[col].str.title() if df[col].dtype == 'object' else df[col]
            
        # 4. Handle missing values
        # Replace empty strings with NaN
        df = df.replace(r'^\s*$', np.nan, regex=True)
        
        # For numeric columns, fill NaN with median
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        for col in numeric_columns:
            df[col] = df[col].fillna(df[col].median())
            
        # For string columns, fill NaN with 'Unknown'
        for col in string_columns:
            df[col] = df[col].fillna('Unknown')
            
        # 5. Remove rows where all values are NaN
        df = df.dropna(how='all')
        
        # Create cleaning report
        cleaning_report = {
            'original_rows': original_rows,
            'cleaned_rows': len(df),
            'rows_removed': original_rows - len(df),
            'missing_values_filled': (df == 'Unknown').sum().sum(),
            'columns_cleaned': original_columns
        }
        
        # Create output filename while preserving the original folder structure
        rel_path = os.path.relpath(file_path, "Data")
        output_path = os.path.join(output_folder, rel_path)
        output_dir = os.path.dirname(output_path)
        
        # Create the necessary subdirectories in the output folder
        os.makedirs(output_dir, exist_ok=True)
        
        # Create output filename
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        cleaned_file_path = os.path.join(output_dir, f'{base_name}_cleaned_{timestamp}.csv')
        
        # Save cleaned file
        df.to_csv(cleaned_file_path, index=False)
        
        return cleaned_file_path, cleaning_report
    
    except Exception as e:
        return None, {'error': str(e)}

def find_csv_files(folder_path):
    """
    Recursively find all CSV files in the given folder and its subfolders
    
    Parameters:
    folder_path (str): Path to the root folder
    
    Returns:
    list: List of paths to CSV files
    """
    csv_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

def clean_data_folder(data_folder="Data", output_folder="Cleaned_Data"):
    """
    Clean all CSV files in the Data folder and its subfolders
    
    Parameters:
    data_folder (str): Path to the Data folder containing CSV files
    output_folder (str): Path to save cleaned CSV files
    """
    # Check if Data folder exists
    if not os.path.exists(data_folder):
        print(f"Error: {data_folder} folder not found!")
        return None
    
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Find all CSV files
    csv_files = find_csv_files(data_folder)
    
    if not csv_files:
        print(f"No CSV files found in {data_folder} or its subfolders")
        return None
    
    # Initialize summary report
    summary_report = {
        'total_files': 0,
        'successful_cleanings': 0,
        'failed_cleanings': 0,
        'cleaning_reports': {}
    }
    
    # Process each CSV file
    for file_path in csv_files:
        rel_path = os.path.relpath(file_path, data_folder)
        print(f"Processing: {rel_path}")
        
        cleaned_path, report = clean_csv_file(file_path, output_folder)
        
        summary_report['total_files'] += 1
        if cleaned_path:
            summary_report['successful_cleanings'] += 1
            summary_report['cleaning_reports'][rel_path] = report
            print(f"Successfully cleaned: {rel_path}")
        else:
            summary_report['failed_cleanings'] += 1
            summary_report['cleaning_reports'][rel_path] = report
            print(f"Failed to clean: {rel_path}")
    
    return summary_report

# Example usage
if __name__ == "__main__":
    # Run the cleaning process
    summary = clean_data_folder()
    
    # Print summary report if cleaning was performed
    if summary:
        print("\nCleaning Summary:")
        print(f"Total files processed: {summary['total_files']}")
        print(f"Successfully cleaned: {summary['successful_cleanings']}")
        print(f"Failed cleanings: {summary['failed_cleanings']}")
        
        # Print detailed reports for each file
        print("\nDetailed Reports:")
        for file_name, report in summary['cleaning_reports'].items():
            print(f"\nFile: {file_name}")
            if 'error' in report:
                print(f"Error: {report['error']}")
            else:
                print(f"Original rows: {report['original_rows']}")
                print(f"Cleaned rows: {report['cleaned_rows']}")
                print(f"Rows removed: {report['rows_removed']}")
                print(f"Missing values filled: {report['missing_values_filled']}")

Processing: Accidents\Accidents_2015.csv


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_6436\2362213477.py:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully cleaned: Accidents\Accidents_2015.csv
Processing: Accidents\Accidents_2016.csv


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_6436\2362213477.py:17: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully cleaned: Accidents\Accidents_2016.csv
Processing: Accidents\Accidents_2017.csv


...

Successfully cleaned: Casualties\Casualties_2017.csv
Processing: Casualties\Casualties_2018.csv
Successfully cleaned: Casualties\Casualties_2018.csv
Processing: Casualties\Causalties_2016.csv
Successfully cleaned: Casualties\Causalties_2016.csv
Processing: Vehicles\Vehicles_2015.csv
Successfully cleaned: Vehicles\Vehicles_2015.csv
Processing: Vehicles\Vehicles_2016.csv
Successfully cleaned: Vehicles\Vehicles_2016.csv
Processing: Vehicles\Vehicles_2017.csv


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_6436\2362213477.py:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully cleaned: Vehicles\Vehicles_2017.csv
Processing: Vehicles\Vehicles_2018.csv


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_6436\2362213477.py:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully cleaned: Vehicles\Vehicles_2018.csv

Cleaning Summary:
Total files processed: 12
Successfully cleaned: 12
Failed cleanings: 0

Detailed Reports:

File: Accidents\Accidents_2015.csv
Original rows: 140056
Cleaned rows: 140056
Rows removed: 0
Missing values filled: 25299

File: Accidents\Accidents_2016.csv
Original rows: 136621
Cleaned rows: 136621
Rows removed: 0
Missing values filled: 8353

File: Accidents\Accidents_2017.csv
Original rows: 129982
Cleaned rows: 129982
Rows removed: 0
Missing values filled: 56286

File: Accidents\Accidents_2018.csv
Original rows: 122635
Cleaned rows: 122635
Rows removed: 0
Missing values filled: 22842

File: Casualties\Casualties_2015.csv
Original rows: 186189
Cleaned rows: 186189
Rows removed: 0
Missing values filled: 0

File: Casualties\Casualties_2017.csv
Original rows: 170993
Cleaned rows: 170993
Rows removed: 0
Missing values filled: 32768

File: Casualties\Casualties_2018.csv
Original rows: 160597
Cleaned rows: 160597
Rows removed: 0
Mi

In [3]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
import sys
from jinja2 import Template
import webbrowser

def generate_html_report(summary_report, output_folder):
    """
    Generate an HTML report from the cleaning summary
    """
    html_template = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>CSV Cleaning Report</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f5f5f5;
            }
            .container {
                max-width: 1200px;
                margin: 0 auto;
                background-color: white;
                padding: 20px;
                border-radius: 8px;
                box-shadow: 0 0 10px rgba(0,0,0,0.1);
            }
            h1, h2 {
                color: #333;
            }
            .summary-box {
                background-color: #f8f9fa;
                border: 1px solid #dee2e6;
                border-radius: 4px;
                padding: 15px;
                margin: 10px 0;
            }
            table {
                width: 100%;
                border-collapse: collapse;
                margin: 15px 0;
            }
            th, td {
                border: 1px solid #dee2e6;
                padding: 8px;
                text-align: left;
            }
            th {
                background-color: #f8f9fa;
            }
            tr:nth-child(even) {
                background-color: #f8f9fa;
            }
            .success {
                color: #28a745;
            }
            .warning {
                color: #ffc107;
            }
            .error {
                color: #dc3545;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>CSV Cleaning Report</h1>
            <div class="summary-box">
                <h2>Overall Summary</h2>
                <p>Total files processed: {{ summary['total_files'] }}</p>
                <p>Successfully cleaned: <span class="success">{{ summary['successful_cleanings'] }}</span></p>
                <p>Failed cleanings: <span class="error">{{ summary['failed_cleanings'] }}</span></p>
                <p>Total rows processed: {{ summary['total_rows_processed'] }}</p>
                <p>Total rows cleaned: {{ summary['total_rows_cleaned'] }}</p>
                <p>Total null values handled: {{ summary['total_null_values'] }}</p>
                <p>Total errors found: {{ summary['total_errors'] }}</p>
            </div>

            <h2>Folders Processed</h2>
            <ul>
            {% for folder in summary['folders_processed'] %}
                <li>{{ folder }}</li>
            {% endfor %}
            </ul>

            <h2>Columns Processed</h2>
            <ul>
            {% for column in summary['unique_columns'] %}
                <li>{{ column }}</li>
            {% endfor %}
            </ul>

            <h2>Detailed File Reports</h2>
            <table>
                <tr>
                    <th>File Name</th>
                    <th>Original Rows</th>
                    <th>Cleaned Rows</th>
                    <th>Null Values</th>
                    <th>Errors Found</th>
                    <th>Status</th>
                </tr>
                {% for file_name, report in summary['cleaning_reports'].items() %}
                <tr>
                    <td>{{ file_name }}</td>
                    {% if 'error' in report %}
                    <td colspan="4">{{ report['error'] }}</td>
                    <td class="error">Failed</td>
                    {% else %}
                    <td>{{ report['original_rows'] }}</td>
                    <td>{{ report['cleaned_rows'] }}</td>
                    <td>{{ report['null_values'] }}</td>
                    <td>{{ report['errors_found'] }}</td>
                    <td class="success">Success</td>
                    {% endif %}
                </tr>
                {% endfor %}
            </table>
        </div>
    </body>
    </html>
    """

    template = Template(html_template)
    
    # Calculate total statistics
    total_rows_processed = sum(report['original_rows'] for _, report in summary_report['cleaning_reports'].items() if 'error' not in report)
    total_rows_cleaned = sum(report['cleaned_rows'] for _, report in summary_report['cleaning_reports'].items() if 'error' not in report)
    total_null_values = sum(report['null_values'] for _, report in summary_report['cleaning_reports'].items() if 'error' not in report)
    total_errors = sum(report['errors_found'] for _, report in summary_report['cleaning_reports'].items() if 'error' not in report)
    
    # Add totals to summary report
    summary_report['total_rows_processed'] = total_rows_processed
    summary_report['total_rows_cleaned'] = total_rows_cleaned
    summary_report['total_null_values'] = total_null_values
    summary_report['total_errors'] = total_errors
    
    # Generate HTML
    html_content = template.render(summary=summary_report)
    
    # Save HTML report
    report_path = os.path.join(output_folder, 'cleaning_report.html')
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    return report_path

def clean_csv_file(file_path, output_folder):
    """
    Clean a single CSV file and save the cleaned version with enhanced error tracking
    """
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        original_columns = df.columns.tolist()
        
        # Store original row count
        original_rows = len(df)
        
        # Count initial null values
        initial_null_count = df.isnull().sum().sum()
        
        # Track errors found
        errors_found = 0
        
        # Basic cleaning operations
        
        # 1. Remove duplicates
        duplicates_removed = len(df) - len(df.drop_duplicates())
        errors_found += duplicates_removed
        df = df.drop_duplicates()
        
        # 2. Strip whitespace from string columns
        string_columns = df.select_dtypes(include=['object']).columns
        whitespace_errors = 0
        for col in string_columns:
            if df[col].dtype == 'object':
                whitespace_errors += df[col].str.strip().ne(df[col]).sum()
                df[col] = df[col].str.strip()
        errors_found += whitespace_errors
        
        # 3. Standardize text case in string columns
        case_errors = 0
        for col in string_columns:
            if df[col].dtype == 'object':
                case_errors += df[col].str.title().ne(df[col]).sum()
                df[col] = df[col].str.title()
        errors_found += case_errors
            
        # 4. Handle missing values
        df = df.replace(r'^\s*$', np.nan, regex=True)
        
        # Count null values after cleaning
        final_null_count = df.isnull().sum().sum()
        
        # For numeric columns, fill NaN with median
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        for col in numeric_columns:
            df[col] = df[col].fillna(df[col].median())
            
        # For string columns, fill NaN with 'Unknown'
        for col in string_columns:
            df[col] = df[col].fillna('Unknown')
            
        # 5. Remove rows where all values are NaN
        all_null_rows = len(df[df.isnull().all(axis=1)])
        errors_found += all_null_rows
        df = df.dropna(how='all')
        
        # Create cleaning report
        cleaning_report = {
            'original_rows': original_rows,
            'cleaned_rows': len(df),
            'rows_removed': original_rows - len(df),
            'null_values': initial_null_count,
            'errors_found': errors_found,
            'columns_cleaned': original_columns
        }
        
        # Create output filename while preserving the original folder structure
        rel_path = os.path.relpath(file_path, "Data")
        output_path = os.path.join(output_folder, rel_path)
        output_dir = os.path.dirname(output_path)
        
        # Create the necessary subdirectories in the output folder
        os.makedirs(output_dir, exist_ok=True)
        
        # Create output filename
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        cleaned_file_path = os.path.join(output_dir, f'{base_name}_cleaned_{timestamp}.csv')
        
        # Save cleaned file
        df.to_csv(cleaned_file_path, index=False)
        
        return cleaned_file_path, cleaning_report
    
    except Exception as e:
        return None, {'error': str(e)}

def find_csv_files(folder_path):
    """
    Recursively find all CSV files in the given folder and its subfolders
    """
    csv_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

def clean_data_folder(data_folder="Data", output_folder="Cleaned_Data"):
    """
    Clean all CSV files in the Data folder and its subfolders with enhanced reporting
    """
    # Check if Data folder exists
    if not os.path.exists(data_folder):
        print(f"Error: {data_folder} folder not found!")
        return None
    
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Find all CSV files
    csv_files = find_csv_files(data_folder)
    
    if not csv_files:
        print(f"No CSV files found in {data_folder} or its subfolders")
        return None
    
    # Initialize summary report
    summary_report = {
        'total_files': 0,
        'successful_cleanings': 0,
        'failed_cleanings': 0,
        'cleaning_reports': {},
        'folders_processed': set(),
        'unique_columns': set()
    }
    
    # Process each CSV file
    for file_path in csv_files:
        rel_path = os.path.relpath(file_path, data_folder)
        print(f"Processing: {rel_path}")
        
        # Add folder to processed folders list
        folder_path = os.path.dirname(rel_path)
        if folder_path:
            summary_report['folders_processed'].add(folder_path)
        
        cleaned_path, report = clean_csv_file(file_path, output_folder)
        
        summary_report['total_files'] += 1
        if cleaned_path:
            summary_report['successful_cleanings'] += 1
            summary_report['cleaning_reports'][rel_path] = report
            # Add columns to unique columns set
            if 'columns_cleaned' in report:
                summary_report['unique_columns'].update(report['columns_cleaned'])
            print(f"Successfully cleaned: {rel_path}")
        else:
            summary_report['failed_cleanings'] += 1
            summary_report['cleaning_reports'][rel_path] = report
            print(f"Failed to clean: {rel_path}")
    
    # Convert sets to sorted lists for JSON serialization
    summary_report['folders_processed'] = sorted(summary_report['folders_processed'])
    summary_report['unique_columns'] = sorted(summary_report['unique_columns'])
    
    # Generate HTML report
    report_path = generate_html_report(summary_report, output_folder)
    print(f"\nHTML report generated at: {report_path}")
    
    # Open the report in the default web browser
    webbrowser.open('file://' + os.path.abspath(report_path))
    
    return summary_report

# Example usage
if __name__ == "__main__":
    # Run the cleaning process
    summary = clean_data_folder()
    
    if summary:
        print("\nCleaning process completed. Check the HTML report for detailed information.")

Processing: Accidents\Accidents_2015.csv


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_6436\4151867325.py:160: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully cleaned: Accidents\Accidents_2015.csv
Processing: Accidents\Accidents_2016.csv


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_6436\4151867325.py:160: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully cleaned: Accidents\Accidents_2016.csv
Processing: Accidents\Accidents_2017.csv


...

Successfully cleaned: Casualties\Casualties_2017.csv
Processing: Casualties\Casualties_2018.csv
Successfully cleaned: Casualties\Casualties_2018.csv
Processing: Casualties\Causalties_2016.csv
Successfully cleaned: Casualties\Causalties_2016.csv
Processing: Vehicles\Vehicles_2015.csv
Successfully cleaned: Vehicles\Vehicles_2015.csv
Processing: Vehicles\Vehicles_2016.csv
Successfully cleaned: Vehicles\Vehicles_2016.csv
Processing: Vehicles\Vehicles_2017.csv


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_6436\4151867325.py:160: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully cleaned: Vehicles\Vehicles_2017.csv
Processing: Vehicles\Vehicles_2018.csv


C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_6436\4151867325.py:160: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Successfully cleaned: Vehicles\Vehicles_2018.csv

HTML report generated at: Cleaned_Data\cleaning_report.html

Cleaning process completed. Check the HTML report for detailed information.
